In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed 
from numba import jit

from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [29]:
# Define the Franke Fumction
def frankeFunction(x,y):
    #noise = np.random.normal(0.5,1,len(x))
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4 #+ noise

# Create the data points in mesh grid form
def createDataPoints():
    x = np.arange(0, 1, 0.1)
    y = np.arange(0, 1, 0.1)
    x_d, y_d = np.meshgrid(x,y)
    z_d = frankeFunction(x_d,y_d)
    return x_d, y_d, z_d

# Create the design matrix with all possible combinations
@jit
def createDesignMatrix(x, y, n):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    p = int((n+1)*(n+2)/2)
    X = np.ones((N,p))

    for i in range(1, n+1):
        q = int(i*(i+1)/2)
        for j in range(i+1):
            X[:,q+j] = (x**(i-j))*(y**j)
    return X

# Predict the outcome using matrix inversion
def predict(X, z_data):
    beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(z_data)
    ztilde = X @ beta
    return ztilde

# Calculate the mean square error (MSE)
def MSE(z_data, z_model):
    n = np.size(z_model)
    return np.sum((z_data-z_model)**2)/n

# Calculate the coefficient of determination (R2)
def R2(z_data, z_model):
    n = np.size(z_data)
    return 1 - np.sum((z_data-z_model)**2)/np.sum((z_data-(np.sum(z_data)/n))**2)

#Scale the data and split them for training and testing
def splitAndScale(x,z):
    X_train, X_test, z_train, z_test = train_test_split(x,z,test_size=0.3)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train, X_test, z_train, z_test

# Calculate the confidence interval of parameters Beta's
def CIBeta(X, Beta):
#     varBeta = np.diagonal(np.linalg.inv(X.T @ X)*sig2)
#     stdBeta = np.sqrt(varBeta)
#     CIlow = Beta - 1.645*stdBeta
#     CIhigh = Beta + 1.645*stdBeta

# I cannot get this function to work with X_train
def SVDinv(A):
    U, s, VT = np.linalg.svd(A)
    print(A.shape)
    print(U.shape)
    print(s.shape)
    print(VT.shape)
    S = np.diag(s)
    UT = np.transpose(U)
    V = np.transpose(VT)
    invS = np.linalg.inv(S)
    #print(np.matmul(V,np.matmul(invD,UT)).shape)
    return np.matmul(V,np.matmul(invS,UT))

def kfoldCV(X,z,k=5):
    #First, split data into test and training set
    x_train, x_test, y_train, y_test = splitAndScale(X,x_d)
    
    data = np.concatenate((x_train, y_train), axis=1)
    n = np.size(data,0)
    m = np.size(data,1) 
    
    #Randomly shuffle the data
    np.random.shuffle(data)
    
    #Split data set into k parts
    splitData = np.array_split(data,k)
    
    for i in range(0,k):
        train = np.zeros((0,3))
        for j in range(0,k):
            if j==i:
                test = splitData[j]
                continue
            train = np.concatenate((train, split_dataset[j]), axis=0)
    

def ridgeRegression():
    pass

def lassoRegression():
    pass


    
    


a)

In [46]:
n = 5 # polynomial degree
x, y, z = createDataPoints()
X = createDesignMatrix(x,y,n)

X_train, X_test, z_train, z_test = splitAndScale(X,np.ravel(z))
# A = np.transpose(X_train) @ X_train
# SVDinvA = SVDinv(A)
z_tilde = predict(X_train, z_train)
print(f"The R2 value for a polynomial of order {n}: {R2(z_train, z_tilde)}")
print(f"The MSE value for a polynomial of order {n}: {MSE(z_train, z_tilde)}")

The R2 value for a polynomial of order 5: 0.9782740665541235
The MSE value for a polynomial of order 5: 0.00194504473327786


b)

In [65]:
x.shape

(10, 10)

In [71]:
kfoldCV(x,1,1,2)

[[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]]
[[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]]
